### This code is used to crop the circular region form the paper assay and extract the raw pixel values.

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,1873):

    ## path to Image dataset directory 
    filename= "C:/Users/Brinda Khanal/Desktop/All pesticide training data/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    #original_image= np.copy(image)
    original_image= cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        (x,y),r = cv2.minEnclosingCircle(c) # get the minimum closing circle
        circles.append([int(x),int(y),int(r*2)]) #increase the circle's radius by a fraction
        print ("cricles:",x,y,r)


    circles= circles[0:2][::-1]# first two circles are what we need     
    mask = np.zeros(original_image.shape,np.uint8) # empty array of image size
    for cr in circles:    
        cv2.circle(mask,(cr[0],cr[1]),cr[2],(255,255,255),-1) # create a circular mask of greater radius than the circular zones

    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask_contour= cv2.findContours(mask_gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnts = mask_contour[1] if cv2.__version__.startswith('3') else mask_contour[0]
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        print ("Box Coordinates",x,y,w,h)
        rectangles.append((x,y,w,h))

    if rectangles[0][0]>rectangles[1][0]:
        rectangles= rectangles[::-1]


    print (rectangles)

    img_ref= original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    img_sample= original_image[rectangles[1][1]:rectangles[1][1]+rectangles[1][3],rectangles[1][0]:rectangles[1][0]+rectangles[1][2]]
    img_ref = cv2.resize(img_ref,(16,16),interpolation=cv2.INTER_AREA)
    img_sample= cv2.resize(img_sample,(16,16),interpolation=cv2.INTER_AREA)
    image_vector= np.concatenate((img_ref,img_sample),axis =2)
    #stacked_img=np.hstack((img_ref,img_sample))
    #image_vector= np.ravel(stacked_img)
    #image_vector= stacked_img
    
    print (image_vector.shape)
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/All pesticide training data/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/new data/downsampled_image_LAB_32_pesticide_train.pkl')
result.head(20)

# Code to extract inner segments 

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,601):
  
    # path to Image Dataset directory 
    filename= "C:/Users/Brinda Khanal/Desktop/Food Color Constant Distance and plane of angle/Combined/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    original_image= np.copy(image)
    
    #original_image= cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        x,y,w,h = cv2.boundingRect(c)
        rectangles.append((x,y,w,h))


    if rectangles[0][0]>rectangles[1][0]:
        rectangles= rectangles[0:2][::-1]


    print (rectangles)

    img_ref= original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    img_sample= original_image[rectangles[1][1]:rectangles[1][1]+rectangles[1][3],rectangles[1][0]:rectangles[1][0]+rectangles[1][2]]
    img_ref = cv2.resize(img_ref,(16,16),interpolation=cv2.INTER_AREA)
    img_sample= cv2.resize(img_sample,(16,16),interpolation=cv2.INTER_AREA)
    #stacked_img=np.hstack((img_ref,img_sample))
    #stacked_img= normalize_intensity(stacked_img)
    image_vector= np.concatenate((img_ref,img_sample),axis =2)
    #image_vector= np.ravel(stacked_img)
    #image_vector= img_sample
    print (image_vector.shape)
   
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/Food Color Constant Distance and plane of angle/Combined/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/new data/downsampled_image_RGB_32_600_inner_circle.pkl')
result.head(20)

# Extract ROI with both Sample and Reference Region

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,1574):
  
    # Path to Image Dataset Directory 
    filename= "C:/Users/Brinda Khanal/Desktop/All images/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    original_image= np.copy(image)
    #original_image= cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        (x,y),r = cv2.minEnclosingCircle(c) # get the minimum closing circle
        circles.append([int(x),int(y),int(r*3)]) #increase the circle's radius by a fraction
        print ("cricles:",x,y,r)


    circles= circles[0:2][::-1]# first two circles are what we need     
    mask = np.zeros(original_image.shape,np.uint8) # empty array of image size
    for cr in circles:    
        cv2.circle(mask,(cr[0],cr[1]),cr[2],(255,255,255),-1) # create a circular mask of greater radius than the circular zones

    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask_contour= cv2.findContours(mask_gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnts = mask_contour[1] if cv2.__version__.startswith('3') else mask_contour[0]
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        print ("Box Coordinates",x,y,w,h)
        rectangles.append((x,y,w,h))


    print (rectangles)
    sample_and_ref = original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    sample_and_ref = cv2.resize(sample_and_ref,(32,32),interpolation=cv2.INTER_AREA)
    #image_vector= np.ravel(sample_and_ref)
    image_vector=sample_and_ref
    
    print (image_vector.shape)
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/All images/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/downsampled_feature_vectors_RGB_both_sample_and_reference_32_CNN.pkl')
result.head(20)

# Extract only the sample zone inner circle

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,201):
  
    # Path to Image Dataset Directory 
    filename= "C:/Users/Brinda Khanal/Desktop/All images 200/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    original_image= np.copy(image)
    
    #original_image= cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        x,y,w,h = cv2.boundingRect(c)
        rectangles.append((x,y,w,h))


    if rectangles[0][0]>rectangles[1][0]:
        rectangles= rectangles[0:2][::-1]


    print (rectangles)

    img_ref= original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    img_sample= original_image[rectangles[1][1]:rectangles[1][1]+rectangles[1][3],rectangles[1][0]:rectangles[1][0]+rectangles[1][2]]
    img_ref = cv2.resize(img_ref,(16,16),interpolation=cv2.INTER_AREA)
    img_sample= cv2.resize(img_sample,(16,16),interpolation=cv2.INTER_AREA)
    #stacked_img=np.hstack((img_ref,img_sample))
    #stacked_img= normalize_intensity(stacked_img)
    #image_vector= np.concatenate((img_ref,img_sample),axis =2)
    image_vector= np.ravel(img_sample)
    #image_vector= stacked_img
    #image_vector = img_sample
    print (image_vector.shape)
   
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/All images 200/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/downsampled_feature_vectors_RGB_inner_circle_sample_only_test_try.pkl')
result.head(20)

# Only sample Zone, pheripheral white space included

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,2353):
  
    # path to Image Dataset Directory 
    filename= "C:/Users/Brinda Khanal/Desktop/All images 2400/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    original_image= np.copy(image)
    #original_image= cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        (x,y),r = cv2.minEnclosingCircle(c) # get the minimum closing circle
        circles.append([int(x),int(y),int(r)]) 
        print ("cricles:",x,y,r)


    circles= circles[0:2]  # first two circles are what we need 
    if circles[0][0]>circles[1][0]:
        circles = circles[::-1]

    cr = circles[1]
    print (cr)
    
    
    mask = np.zeros(original_image.shape,np.uint8) # empty array of image size
    cv2.circle(mask,(cr[0],cr[1]),cr[2]*3,(255,255,255),-1) # create a circular mask of greater radius than the circular zones

    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask_contour= cv2.findContours(mask_gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnts = mask_contour[1] if cv2.__version__.startswith('3') else mask_contour[0]
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        print ("Box Coordinates",x,y,w,h)
        rectangles.append((x,y,w,h))
    print (rectangles)

    img_sample= original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    img_sample= cv2.resize(img_sample,(16,16),interpolation=cv2.INTER_AREA)
    image_vector = img_sample
    #image_vector= np.concatenate((img_ref,img_sample),axis =2)
    #stacked_img=np.hstack((img_ref,img_sample))
    #image_vector= np.ravel(stacked_img)
    #image_vector= stacked_img
    
    print (image_vector.shape)
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/All images 2400/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/downsampled_feature_vectors_RGB_CNN_sample_only_2400.pkl')
result.head(20)

# Normalize each image idependently  

This code is used to nomralize each image independently. It is not required to run. We just used this to test some experiments

In [ ]:
# segment sample zone and reference zone, and then downsample them to 16*16 image each.
# stack two image horizontally to form 32*16 image
# extract 3 color values (RGB, HSV or LAB) of each pixel and flatten the final image into (1,32*16*3) feature vector 
# repeat the process for all the images (IMG_1.jpg, IMG_2.jpg,....), push the features into pandas dataframe
# save dataframe to a pickle file

# just change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2HSV) for HSV values
# change image_for_plot= cv2.cvtColor(image,cv2.COLOR_BGR2LAB) for Lab values
# also, rename dataframe coulmn index accordingly

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd 

df= pd.DataFrame(columns=['ImageName', 'Feature Vector'])
for i in range (1,201):
  
    # path to Image Dataset Directory 
    filename= "C:/Users/Brinda Khanal/Desktop/All images 200/IMG_"+str(i)+".jpg"
    print (filename) 
    
    image_name= "IMG_"+str(i)

    # read the image
    image = cv2.imread(filename)
    original_image= np.copy(image)
    #original_image= cv2.cvtColor(image,cv2.COLOR_BGR2LAB)
    
    #image = cv2.medianBlur(image,5)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_mean= np.mean(gray)
    print (gray_mean)

    #formula to choose right lower binary threshold, self-estimated formula after observance 
    # T= 0.8I-30
    lower_th= 0.8*gray_mean-30

    ret2,th2 = cv2.threshold(gray,lower_th,255,cv2.THRESH_BINARY)
    
    kernel = np.ones((5,5),np.uint8)
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel)
    
    # find the contour 
    contours= cv2.findContours(th2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
    cnts = contours[1] if cv2.__version__.startswith('3') else contours[0]
    rectangles = []
    circles=[]

    for c in cnts:

        if cv2.contourArea(c) < 2000 :
            continue
        (x,y),r = cv2.minEnclosingCircle(c) # get the minimum closing circle
        circles.append([int(x),int(y),int(r*2)]) #increase the circle's radius by a fraction
        print ("cricles:",x,y,r)


    circles= circles[0:2][::-1]# first two circles are what we need     
    mask = np.zeros(original_image.shape,np.uint8) # empty array of image size
    for cr in circles:    
        cv2.circle(mask,(cr[0],cr[1]),cr[2],(255,255,255),-1) # create a circular mask of greater radius than the circular zones

    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask_contour= cv2.findContours(mask_gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnts = mask_contour[1] if cv2.__version__.startswith('3') else mask_contour[0]
    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
        print ("Box Coordinates",x,y,w,h)
        rectangles.append((x,y,w,h))

    if rectangles[0][0]>rectangles[1][0]:
        rectangles= rectangles[::-1]


    print (rectangles)

    img_ref= original_image[rectangles[0][1]:rectangles[0][1]+rectangles[0][3],rectangles[0][0]:rectangles[0][0]+rectangles[0][2]]
    img_sample= original_image[rectangles[1][1]:rectangles[1][1]+rectangles[1][3],rectangles[1][0]:rectangles[1][0]+rectangles[1][2]]
    img_ref = cv2.resize(img_ref,(16,16),interpolation=cv2.INTER_AREA)
    img_sample= cv2.resize(img_sample,(16,16),interpolation=cv2.INTER_AREA)
    means = img_ref.mean(axis=(0,1), dtype='float64')
    stds = img_sample.std(axis=(0,1), dtype='float64')
    stacked_img = (img_ref-means)/stds
    #image_vector= np.concatenate((img_ref,img_sample),axis =2)
    #stacked_img=np.hstack((img_ref,img_sample))
    image_vector= np.ravel(stacked_img)
    #image_vector= stacked_img
    
    print (image_vector.shape)
    df= df.append({'ImageName': image_name,'Feature Vector': image_vector}, ignore_index=True)
    
Label_data= pd.read_excel("C:/Users/Brinda Khanal/Desktop/All images 200/Labels.xlsx")
result = df.merge(Label_data, on ='ImageName')  

    
result.to_pickle('C:/Users/Brinda Khanal/Documents/Bidur Git Repo/padColorimetry/Data/downsampled_feature_vectors_RGB_normalize_within_test_try.pkl')
result.head(20)